# Anomaly Detection with PCA

Load an anomaly detection dataset and apply the PCA algorithm to reduce the dimensionality. Train an anomaly detection model using the reduced dataset. Classify the dataset using the trained model and print out the rows classified as anomalies. Save the anomaly detection model to anomaly_detection.mlnet. Once completed, print 'task resolved' in the end.

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

var mlContext = new MLContext(seed: 0);

// Load your data
// This is where you need to load your anomaly detection dataset
// For the sake of this example, I'm using a simple list of DataPoints
var samples = new List<DataPoint>()
{
    new DataPoint(){ Features = new float[3] {0, 2, 1} },
    new DataPoint(){ Features = new float[3] {0, 2, 1} },
    new DataPoint(){ Features = new float[3] {0, 2, 1} },
    new DataPoint(){ Features = new float[3] {0, 1, 2} },
    new DataPoint(){ Features = new float[3] {0, 2, 1} },
    new DataPoint(){ Features = new float[3] {2, 0, 0} }
};

var data = mlContext.Data.LoadFromEnumerable(samples);

// Apply PCA
var pipeline = mlContext.AnomalyDetection.Trainers.RandomizedPca(
    featureColumnName: nameof(DataPoint.Features), rank: 1,
        ensureZeroMean: false);

// Train the model
var model = pipeline.Fit(data);

// Apply the trained model on the training data
var transformed = model.Transform(data);

// Read ML.NET predictions into IEnumerable<Result>
var results = mlContext.Data.CreateEnumerable<Result>(transformed,
    reuseRowObject: false).ToList();

// Print out the rows classified as anomalies
for (int i = 0; i < samples.Count; ++i)
{
    var result = results[i];
    var featuresInText = string.Join(',', samples[i].Features);

    if (result.PredictedLabel)
        Console.WriteLine($"The {i}-th example with features [{featuresInText}] is an outlier with a score of being inlier {result.Score}");
}

// Save the model
mlContext.Model.Save(model, data.Schema, "anomaly_detection.mlnet");

Console.WriteLine("task resolved");

class DataPoint
{
    [VectorType(3)]
    public float[] Features { get; set; }
}

class Result
{
    public bool PredictedLabel { get; set; }
    public float Score { get; set; }
}